In [1]:
from collections import defaultdict
import datetime
from gensim import corpora, models, similarities
from matplotlib import pyplot as plt
import numpy as np
import operator
import pandas as pd
import pickle
import random
import re
import scipy
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm_notebook

import src.knntools as knntools
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools
import src.tools as tools
import src.recencytools as recency
import src.scoring as scoring
import src.textembeddingtools as texttools

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        sep=',', parse_dates=True, header=0)
path_to_results = 'results/'

In [3]:
train_info, train_email_ids_per_sender, val_info, val_email_ids_per_sender = scoring.get_train_val(training, training_info, train_frac=0.95)

Processing training !

Processing val !



In [4]:
train_body_dict = preprocess.body_dict_from_panda(train_info)
val_body_dict = preprocess.body_dict_from_panda(val_info)


In [5]:
train_token_dict = texttools.get_token_dict(train_body_dict)
val_token_dict = texttools.get_token_dict(val_body_dict)

In [6]:
train_email_list, train_idx_to_mids = texttools.token_dicts_to_token_lists(train_token_dict, rarity_threshold=3)
val_email_list, val_idx_to_mids = texttools.token_dicts_to_token_lists(val_token_dict, rarity_threshold=3)

In [7]:
# Assign unique id to word
train_word_id_dic = corpora.Dictionary(train_email_list)

# Compute email corpush as bow [[(wordid_1_1, count_1_1), ...] ...]
train_email_corpus = [train_word_id_dic.doc2bow(text) for text in train_email_list]
val_email_corpus = [train_word_id_dic.doc2bow(text) for text in val_email_list]

In [8]:
nb_topics = 200
nb_emails = len(train_email_corpus)
print(nb_emails)

41432


In [9]:
# model_vars_path = 'variables/hdp_similarities_{thres_nb}_words_out_no_punct'.format(thres_nb=rarity_thres)
stored_model_results = ['variables/train-lda-200.p']

lda_model = texttools.compute_model(train_email_corpus, train_word_id_dic, model='lda', nb_topics=nb_topics,
                                    use_saved=True, save=True, model_results_path=stored_model_results[0])

In [11]:
# train_senders_mid_features_dict, train_senders_idx_to_mid_dic = texttools.get_sender_model_features_from_tokens(train_email_ids_per_sender,
#                                                     train_token_dict, train_word_id_dic, lda_model, nb_topics)

In [18]:
train_lda_sender_features_path = 'variables/train-lda-200-sender-features-dic.p'

# with open(train_lda_sender_features_path, 'wb') as outfile:
#     pickle.dump(train_senders_mid_features_dict, outfile)
    
with open(train_lda_sender_features_path, 'rb') as infile:
    train_senders_mid_features_dict = pickle.load(infile)

train_lda_sender_idx_to_mid_path = 'variables/train-lda-200-sender-idx-to-mids-dic.p'

# with open(train_lda_sender_idx_to_mid_path, 'wb') as outfile:
#     pickle.dump(train_senders_idx_to_mid_dic, outfile)
    
with open(train_lda_sender_idx_to_mid_path, 'rb') as infile:
    train_senders_idx_to_mid_dic = pickle.load(infile)

In [19]:
val_senders_mid_features_dict, val_senders_idx_to_mid_dic = texttools.get_sender_model_features_from_tokens(val_email_ids_per_sender, val_token_dict,
                                                                                train_word_id_dic, lda_model, nb_topics)

In [20]:
train_day_features_dic = recency.get_sender_sparse_date_info(train_email_ids_per_sender, 
                                                         train_senders_idx_to_mid_dic, train_info)
val_day_features_dic = recency.get_sender_sparse_date_info(val_email_ids_per_sender,
                                                       val_senders_idx_to_mid_dic, val_info)

In [21]:
train_stacked_features_dict = texttools.create_stacked_feature_dic([train_senders_mid_features_dict, train_day_features_dic])
val_stacked_features_dict = texttools.create_stacked_feature_dic([val_senders_mid_features_dict, val_day_features_dic])

In [22]:
train_sender_recipients_binaries, train_sender_idx_to_recipients = preprocess.get_one_hot_sender_recipients(train_senders_idx_to_mid_dic, train_info)

In [23]:
val_sender_recipients_binaries, val_sender_idx_to_recipients = preprocess.get_one_hot_sender_recipients(val_senders_idx_to_mid_dic, val_info)

In [31]:
tree_nbs = [50, 100]
scores = []
for nb_tree in tree_nbs:
    val_tree_predictions = texttools.tree_train_predict(train_stacked_features_dict, train_sender_recipients_binaries,
                           train_sender_idx_to_recipients, val_stacked_features_dict, val_senders_idx_to_mid_dic, nb_tree=nb_tree)
    score = scoring.compute_prediction_mad(val_tree_predictions, val_info)
    scores.append(score)
    print(score)


0.332141113982


In [29]:
current_score = scoring.compute_prediction_mad(val_tree_predictions, val_info)

In [30]:
current_score

0.29496893072474467

In [32]:
path_to_trees = 'ranks_val/trees-tree-nb-{nb_tree}'.format(nb_tree=nb_tree)
with open(path_to_trees, 'wb') as infile:
          pickle.dump(val_tree_predictions, infile)

